In [1]:
import numpy as np
import cv2
import os
import re
from features_description import *
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
f2, f3, f4 = describe_features((19,19))

In [4]:
print(f2.shape)
print(f3.shape)
print(f4.shape)

print(f2.shape[0] + f3.shape[0] + f4.shape[0])

(34200, 2, 4)
(21660, 3, 4)
(8100, 4, 4)
63960


# Read dataset 

In [5]:
faces_path = 'face.train/train/face'
non_faces_path = 'face.train/train/non-face'
face_img_names = os.listdir(faces_path)
non_face_img_names = os.listdir(non_faces_path)
for i in range(len(face_img_names)):
    face_img_names[i] = faces_path + '/' + face_img_names[i]
for i in range(len(non_face_img_names)):
    non_face_img_names[i] = non_faces_path + '/' + non_face_img_names[i]
face_img_names = np.array(face_img_names)
non_face_img_names = np.array(non_face_img_names)

In [6]:
def get_dataset(face_names, non_face_names):
    # return np array of (img, label)
    imgs, labels = [], []
    for i in range(len(face_names)):
        img = cv2.imread(face_names[i], 0)
        img = cv2.resize(img, (19, 19))
        imgs.append(img)
        labels.append(1)
    for i in range(len(non_face_names)):
        img = cv2.imread(non_face_names[i], 0)
        img = cv2.resize(img, (19, 19))
        imgs.append(img)
        labels.append(-1)
    return np.array(imgs), np.array(labels)

dataset = get_dataset(face_img_names, non_face_img_names)
dataset_ii = getIntegralImage(dataset[0]), dataset[1]
print(dataset_ii[0].shape, dataset_ii[1].shape)

(6977, 20, 20) (6977,)


In [7]:
class Features(Dataset):
    def __init__(self, f):
        if type(f) == list or type(f) == np.ndarray:
            self.f = torch.from_numpy(np.array(f)).type(torch.int64)
        else:
            self.f = f.type(torch.int64)

    def __len__(self):
        return self.f.shape[0]
    def __getitem__(self, idx):
        return self.f[idx]

f2d = Features(f2)
f3d = Features(f3)
f4d = Features(f4)



bs = 1000
f2d_loader = DataLoader(f2d, batch_size=bs, shuffle=False)
f3d_loader = DataLoader(f3d, batch_size=bs, shuffle=False)
f4d_loader = DataLoader(f4d, batch_size=bs, shuffle=False)


In [8]:
iis = dataset_ii[0].astype(int)
iis = torch.from_numpy(iis).to(device)
ii_f2 = torch.zeros((iis.shape[0], f2.shape[0]))
ii_f3 = torch.zeros((iis.shape[0], f3.shape[0]))
ii_f4 = torch.zeros((iis.shape[0], f4.shape[0]))
for i, f2_b in enumerate(f2d_loader):
    f2_b = f2_b.to(device)
    ii_f2[:, i*bs:(i+1)*bs] = (getFeatureValue(iis, f2_b[:, 0]) - getFeatureValue(iis, f2_b[:,1])).to('cpu')

for i, f3_b in enumerate(f3d_loader):
    f3_b = f3_b.to(device)
    ii_f3[:, i*bs:(i+1)*bs] = (-getFeatureValue(iis, f3_b[:, 0]) + getFeatureValue(iis, f3_b[:,1]) - getFeatureValue(iis, f3_b[:,2])).to('cpu')

for i, f4_b in enumerate(f4d_loader):
    f4_b = f4_b.to(device)
    ii_f4[:, i*bs:(i+1)*bs] = (-getFeatureValue(iis, f4_b[:, 0]) + getFeatureValue(iis, f4_b[:,1]) + getFeatureValue(iis, f4_b[:,2]) - getFeatureValue(iis, f4_b[:,3])).to('cpu')

In [9]:
print(ii_f2.shape)
print(ii_f3.shape)
print(ii_f4.shape)


torch.Size([6977, 34200])
torch.Size([6977, 21660])
torch.Size([6977, 8100])


In [10]:
print(ii_f2[:,0:3])


tensor([[ -4.,   2.,   1.],
        [ 44., -45., 104.],
        [  1.,  -6.,   9.],
        ...,
        [  0.,  -1.,   1.],
        [ -1.,  -8.,  -4.],
        [  2.,   0.,  44.]])


In [11]:
print(f3[1])
print(ii_f3[:,0:3])


[[0 0 1 1]
 [1 0 2 1]
 [2 0 3 1]]
tensor([[ -25.,  -25.,  -85.],
        [ -71.,  -55., -189.],
        [ -28.,  -44., -119.],
        ...,
        [  -1.,    0.,   -1.],
        [-115., -177., -229.],
        [ -13.,   -5., -282.]])


In [12]:
print(f4[0])
print(ii_f4[:,0:3])


[[0 0 1 1]
 [0 1 1 2]
 [1 0 2 1]
 [1 1 2 2]]
tensor([[ -1.,   3.,   5.],
        [ 40.,  81., 110.],
        [ -3.,   4.,  76.],
        ...,
        [  1.,   1.,   1.],
        [ -1.,  -6.,  -8.],
        [ -1.,  -1.,   4.]])


In [15]:
all_features = torch.cat((ii_f2, ii_f3, ii_f4), dim=1)
all_features = all_features.t()
print(all_features.shape)
all_features = all_features.numpy()
# save to npy
np.savez_compressed('all_features.npz', all_features)
# np.save('all_features.npy', all_features)

torch.Size([63960, 6977])


In [29]:
print(dataset[0][0],'\n', dataset[0][1])

[[ 25  21  21  26  38  48  41  82 119 137 140 145 150 151 147 143 137 123
  106]
 [ 23  20  19  22  43  37  67  98 127 137 138 141 148 149 147 144 134 122
  116]
 [ 23  21  18  21  25  27  43  81 104 128 124  94  81  66  72  80  95 115
  119]
 [ 22  34  27  44  50  36  32  48  69 112 100  62  47  35  41  50  55  63
  110]
 [ 53  74  66  44  35  29  44  48  74 128 117  61  53  53  47  59  77  91
  107]
 [ 73  82  70  62  60  55  78  61  76 138 125  87  77  72  43  65  73 102
  114]
 [ 84  90  86  85  91  94  86  75  93 139 133 109  98  92  93 104 111 121
  120]
 [ 89  97 103 110 111 108  99  90  98 138 135 125 129 125 122 130 134 134
  128]
 [ 90  98 113 121 120 114 106  90 103 137 134 124 138 140 142 143 144 141
  127]
 [ 89  95 108 120 122 115 102  90 118 144 141 119 127 140 146 146 144 137
  123]
 [ 88  92 104 118 123 110  92  79 100 116 102 102 120 136 144 144 141 131
  116]
 [ 87  89 101 115 121 107  89  68  69  70  84  89 133 138 144 142 133 124
  110]
 [ 88  85  94 110 110  96  8